In [2]:
import mediapipe as mp
import cv2
import numpy as np

In [3]:
from PIL import ImageFont, ImageDraw, Image
import numpy as np

def draw_text(img, text, position, font_size, font_color):
    font_path = "C:/Windows/Fonts/gulim.ttc"  # Windows에서 Gulim 폰트 경로

    # opencv 이미지를 PIL이미지로 변환
    img_pil = Image.fromarray(img)

    # PIL Draw 객체 생성
    draw = ImageDraw.Draw(img_pil)

    # 폰트 스타일 지정
    font = ImageFont.truetype(font_path, font_size)

    # PIL 이미지에 텍스트 입력
    draw.text(position, text, font=font, fill=font_color)
    return np.array(img_pil) # 최종 numpy array 로 이미지 형태 반환

In [4]:
# MediaPipe Pose 설정
pose = mp.solutions.pose.Pose(
    static_image_mode=False,        # 입력 이미지가 정적 이미지인지, 비디오 스트림 프레임인지 설정(False인 경우 비디오 스트림으로 처리)
    model_complexity=1,             # 0,1,2 의 순으로 정확도가 올라가는 대신 속도가 느림
    smooth_landmarks=True,          # 프레임간의 랜드 마크 위치를 부드럽게 처리할 지 여부 결정
    enable_segmentation=True,       # 포즈 검출과 인물 마스크를 생성할지 여부 결정. 배경 제거 등의 용도로 사용
    smooth_segmentation=True,       # 인물 마스크의 경계를 부드럽게 처리
    min_detection_confidence=0.5,   # 사람 검출 최소 신뢰도 임계값
    min_tracking_confidence=0.5     # 포즈 추적의 신뢰도 임계값
)

# 그리기 함수 초기화
mp_drawing = mp.solutions.drawing_utils

#####################################################################################################


def get_pose(pose_landmarks):

    # 왼쪽 어깨(shoulder) 11 // 오른쪽 어깨(shoulder) 12
    left_shoulder, right_shoulder = pose_landmarks.landmark[11], pose_landmarks.landmark[12]
    
    # 왼쪽 팔꿈치(elbow) 13 // 오른쪽 팔꿈치(elbow) 14
    left_elbow, right_elbow = pose_landmarks.landmark[13], pose_landmarks.landmark[14]
    
    # 왼쪽 손목(wrist) 15 // 오른쪽 손목(wrist) 16
    left_wrist, right_wrist = pose_landmarks.landmark[15], pose_landmarks.landmark[16]

    # 왼쪽 손바닥(palm) 19 // 오른쪽 손바닥(palm) 20
    left_palm, right_palm = pose_landmarks.landmark[19], pose_landmarks.landmark[20]

    # 입
    mouth = pose_landmarks.landmark[10]

    # 코
    nose = pose_landmarks.landmark[0]

                                                                                             
    b_Next_Level_Left = (left_elbow.x < left_shoulder.x     and     # 팔꿈치.x      < 왼쪽 어깨.x
                         left_wrist.y < left_shoulder.y     and     # 왼쪽 손목.y   < 왼쪽 팔꿈치.y 
                         left_palm.y < left_elbow.y         and     # 왼쪽 손바닥.y > 왼쪽 팔꿈치.y
                         left_palm.x < left_shoulder.x      and     # 왼쪽 손바닥.x < 왼쪽 어깨.x
                         left_palm.x > right_shoulder.x     and     # 왼쪽 손바닥.x > 오른어깨.x
                         left_palm.x > mouth.x                      # 왼쪽 손바닥.x > 입.x
                         )
    
    # b_Next_Level_Right = right_elbow.x > right_shoulder

    b_victory = (left_wrist.y < nose.y              and             # 왼쪽 손목.y   < 코.y
                 right_wrist.y < nose.y             and             # 오른쪽 손목.y < 코.Y
                 left_wrist.x > right_wrist.x                       # 왼쪽 손목.x   > 오른쪽 손목.x
                 )
    
    b_x = (left_wrist.y < left_shoulder.y           and             # 왼쪽 손목.y   < 왼쪽 어깨.y
              right_wrist.y < right_shoulder.y      and             # 오른쪽 손목.y < 오른쪽 어깨.y
              left_wrist.x < right_wrist.x                          # 왼쪽 손목.x   > 오른쪽 손목.x
              )
    
    b_armCross = (left_wrist.y < left_elbow.y       and             # 왼쪽 손목.y   < 왼쪽 팔꿈치.y
                  right_wrist.y < right_elbow.y     and             # 오른쪽 손목.y < 오른쪽 팔꿈치.y
                  left_wrist.x < left_elbow.x       and             # 왼쪽 손목.y   < 왼쪽 팔꿈치.y
                  right_wrist.x > right_elbow.x     and             # 오른쪽 손목.y < 오른쪽 팔꿈치.y
                  left_wrist.x < right_wrist.x                      # 왼쪽 손목.x   > 오른쪽 손목.x
                  )
    
    # 포즈 판별
    if b_Next_Level_Left:
        return "Next Level"
    elif b_victory:
        return "만세"
    # elif b_x:
    #     return "엑스"
    # elif b_armCross:
    #     return "팔짱"
    else:
        return "모름"
        



#####################################################################################################

# 웹캠 캡처
cap = cv2.VideoCapture(0)

f_count = 0
max_count = 100
score = 0
answer_already = False
game_over = False
pose_list = ['Next Level']
target_pose = np.random.choice(pose_list)

correct_pose_coordinates = {
    "left_shoulder": [] ,
    "right_shoulder": [],
    "left_elbow": [],
    "right_elbow": [],
    "left_wrist": [],
    "right_wrist": [],
    "left_palm": [],
    "right_palm": [],
    "mouth": [],
    "face": []
}

while cap.isOpened():
    # 이미지 읽기
    ret, image = cap.read()
    if not ret:
        break

    # 이미지 변환
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 포즈 추정
    results = pose.process(image)

    # 추정된 포즈 정보 확인
    if results.pose_landmarks:

        # 연결된 랜드마크 시각화
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp.solutions.pose.POSE_CONNECTIONS)
        
#####################################################################################################


        # 정답을 맞춘 경우 정답을 맞춘 순간의 포즈 좌표값을 딕셔너리에 추가
        if not answer_already and not game_over and get_pose(results.pose_landmarks) == target_pose:
            for part, landmark_id in zip(correct_pose_coordinates.keys(), [11, 12, 13, 14, 15, 16, 19, 20, 10, 0]):
                landmark = results.pose_landmarks.landmark[landmark_id]
                correct_pose_coordinates[part].append((landmark.x, landmark.y, landmark.z))



#####################################################################################################

        # # 감지된 포즈의 실시간 좌표를 화면에 표시
        # for idx, landmark in enumerate(results.pose_landmarks.landmark):
        #     height, width, _ = image.shape
        #     cx, cy = int(landmark.x * width), int(landmark.y * height)
        #     cv2.putText(image, f'{idx}', (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

#####################################################################################################

        # 포즈 판별 및 출력
        pose_result = get_pose(results.pose_landmarks)

        image = draw_text(image, str(score), (30, 50), 30, (0, 255, 0)) # 점수
        if game_over:
            image = draw_text(image, "게임 오버...", (image.shape[1] / 2 - 50, 50), 30, (255, 0, 0))
            image = draw_text(image, "잠시 후 종료됩니다.", (image.shape[1] / 2 - 100, 100), 30, (255, 0, 0))
        elif answer_already:
            image = draw_text(image, "정답!", (image.shape[1] / 2 - 25, 50), 30, (255, 255, 255)) # 정답 노출
        else:
            image = draw_text(image, target_pose, (image.shape[1] / 2 - 25, 50), 30, (255, 255, 255)) # 목표 포즈

        if False == answer_already and False == game_over and pose_result == target_pose:
            score += 1
            answer_already = True
           
           # 정답을 맞췄을 때의 포즈 좌표값을 딕셔너리에 추가
            for part, landmark_id in zip(correct_pose_coordinates.keys(), [11, 12, 13, 14, 15, 16, 19, 20, 10, 0]):
                landmark = results.pose_landmarks.landmark[landmark_id]
                correct_pose_coordinates[part].append((landmark.x, landmark.y, landmark.z))



    # 세그멘테이션 마스크 추출 및 배경 제거
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # 이미지 출력
    cv2.imshow('MediaPipe Pose', image)

    # 카운팅 처리
    f_count += 1
    if f_count >= max_count:
        if game_over:
            break
        f_count = 0
        if answer_already:
            if max_count > 40:
                max_count -= 5
            target_pose = np.random.choice(pose_list)
            answer_already = False
        else:
            game_over = True
            max_count = 100

    # 키 입력 처리
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break


#####################################################################################################

print( correct_pose_coordinates )
cap.release()
cv2.destroyAllWindows()

{'left_shoulder': [(0.6739152669906616, 0.7924026846885681, -0.5371907949447632), (0.6739152669906616, 0.7924026846885681, -0.5371907949447632), (0.6363169550895691, 0.8027652502059937, -0.5690995454788208), (0.6363169550895691, 0.8027652502059937, -0.5690995454788208)], 'right_shoulder': [(0.3435831665992737, 0.7876482009887695, -0.14606474339962006), (0.3435831665992737, 0.7876482009887695, -0.14606474339962006), (0.3385797441005707, 0.8085834383964539, -0.20266211032867432), (0.3385797441005707, 0.8085834383964539, -0.20266211032867432)], 'left_elbow': [(0.6096357107162476, 0.8007962703704834, -1.1025969982147217), (0.6096357107162476, 0.8007962703704834, -1.1025969982147217), (0.5858227014541626, 0.7713622450828552, -0.7410082817077637), (0.5858227014541626, 0.7713622450828552, -0.7410082817077637)], 'right_elbow': [(0.19371946156024933, 0.9748017191886902, -0.31153231859207153), (0.19371946156024933, 0.9748017191886902, -0.31153231859207153), (0.2460446059703827, 0.954521358013153

In [29]:
import json

# JSON 형식으로 변환할 데이터 생성
formatted_pose_coordinates = {}

for part, coordinates_list in correct_pose_coordinates.items():
    formatted_pose_coordinates[part] = []
    for x, y, z in coordinates_list:
        formatted_pose_coordinates[part].append({"x": x, "y": y, "z": z})

# JSON 파일로 저장
with open('correct_pose_coordinates.json', 'w') as json_file:
    json.dump(formatted_pose_coordinates, json_file, indent=4)